In [3]:
# Colab cell
try:
    import google.colab
    IN_COLAB = True
    !wget -q https://raw.githubusercontent.com/Competencia-de-Climate-Change/WeatherBench/master/src/ml_training.py
    !pip install -q -U zarr
    !pip install -q -U fsspec
    !pip install -q -U gcsfs
    !pip install -q -U xarray
except:
    IN_COLAB = False

In [21]:
import xarray as xr
import fsspec

ds = xr.open_zarr(fsspec.get_mapper('gcs://weather-bench/region_zarr/'), consolidated=True)

print("Tamanño en GB:", ds.nbytes / 1e9)

_call non-retriable exception: 
Traceback (most recent call last):
  File "/opt/tljh/user/lib/python3.7/site-packages/gcsfs/core.py", line 487, in _call
    validate_response(r, path)
  File "/opt/tljh/user/lib/python3.7/site-packages/gcsfs/core.py", line 132, in validate_response
    raise HttpError({"code": r.status_code})
gcsfs.utils.HttpError


HttpError: 

In [5]:
ds

<xarray.Dataset>
Dimensions:    (lat: 12, level: 13, lon: 14, time: 350640)
Coordinates:
  * lat        (lat) float64 -70.31 -64.69 -59.06 ... -19.69 -14.06 -8.438
  * level      (level) int32 50 100 150 200 250 300 400 500 600 700 850 925 1000
  * lon        (lon) float64 247.5 253.1 258.8 264.4 ... 303.8 309.4 315.0 320.6
  * time       (time) datetime64[ns] 1979-01-01 ... 2018-12-31T23:00:00
Data variables:
    lat2d      (lat, lon) float64 dask.array<chunksize=(12, 14), meta=np.ndarray>
    lon2d      (lat, lon) float64 dask.array<chunksize=(12, 14), meta=np.ndarray>
    lsm        (lat, lon) float32 dask.array<chunksize=(12, 14), meta=np.ndarray>
    orography  (lat, lon) float32 dask.array<chunksize=(12, 14), meta=np.ndarray>
    pv         (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    q          (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    r          (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    slt        (lat, lon) float32 dask.array<chunksize=(12, 14), meta=np.ndarray>
    t          (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    t2m        (time, lat, lon) float32 dask.array<chunksize=(43830, 2, 4), meta=np.ndarray>
    tcc        (time, lat, lon) float32 dask.array<chunksize=(43830, 2, 4), meta=np.ndarray>
    u10        (time, lat, lon) float32 dask.array<chunksize=(43830, 2, 4), meta=np.ndarray>
    v10        (time, lat, lon) float32 dask.array<chunksize=(43830, 2, 4), meta=np.ndarray>
    vo         (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    z          (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2020-03-08 20:51:24 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

# Tree based methods for $Z500$, $T850$ and $T2M$

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
import pickle

from src.ml_training import *

# from dask_ml.wrappers import ParallelPostFit

def to_pickle(obj, fn):
    with open(fn, 'wb') as f:
        pickle.dump(obj, f)
def read_pickle(fn):
    with open(fn, 'rb') as f:
        return pickle.load(f)
    
results = 'results'
import os
if not os.path.exists(results):
    os.makedirs(results)

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  This is separate from the ipykernel package so we can avoid doing imports until


## Data Loading

In [7]:
z_500 = ds.z.sel(level=500).drop('level')
t_850 = ds.t.sel(level=850).drop('level')
t2m = ds.t2m

data = xr.merge([z_500, t_850, t2m])

data_train = data.sel(time=slice('2010', '2016'))
data_test = data.sel(time=slice('2017', '2018'))

z_test = load_test_data('z', ds=ds)
t_test = load_test_data('t', ds=ds)
t2m_test = load_test_data('t2m', ds=ds)

test_data = xr.merge([z_test, t_test, t2m_test])

_, nlat, nlon = data_train.z.shape; nlat, nlon

Selecting from ds...
Selecting from ds...


(12, 14)

In [8]:
# Compute normalization statistics
# Let's only take a sample of the time to speed this up.
data_mean = data_train.isel(time=slice(0, None, 10000)).mean().load()
data_std = data_train.isel(time=slice(0, None, 10000)).std().load()


# Normalize datasets
data_train = (data_train - data_mean) / data_std
data_test = (data_test - data_mean) / data_std

### Define combinations of training data

In [10]:
# Inputs and Outputs
experiments = [
    [['t'],             ['t']],
    [['z', 't'],        ['t']],
    [['z', 't', 't2m'], ['t']],
    [['t2m'],           ['t2m']],
    [['z', 't'],        ['t2m']],
    [['z', 't', 't2m'], ['t2m']],
]

lead_time = 3*24 # Predict 3 days from current time
lead_time_str = '3d'

verbose = False

In [11]:
from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import GridSearchCV

from sklearn.multioutput import MultiOutputRegressor

## Run Experiments

In [12]:
preds  = {
    'dt_reg' : [],
    'rf_reg' : [],
#     'gb_reg' : [],
#     'cv_gb'  : [],
#     'xgb_reg': [],
}

models = {
    'dt_reg' : [],
    'rf_reg' : [],
#     'gb_reg' : [],
#     'cv_gb'  : [],
#     'xgb_reg': [],
}

train_mse = {
    'dt_reg' : [],
    'rf_reg' : [],
#     'gb_reg' : [],
#     'cv_gb'  : [],
#     'xgb_reg': [],
}

test_mse = {
    'dt_reg' : [],
    'rf_reg' : [],
#     'gb_reg' : [],
#     'cv_gb'  : [],
#     'xgb_reg': [],
}

weighted_mse = {
    'dt_reg' : [],
    'rf_reg' : [],
#     'gb_reg' : [],
#     'cv_gb'  : [],
#     'xgb_reg': [],
}


# specify parameters and distributions to sample from
param_grid = {'max_depth': np.arange(3, 100, 30),
              'random_state': 0,
              'n_estimators': [10, 50, 100, 250, 500],
              'learning_rate': [0.01, 0.1, 0.3, 0.5],
              'max_features': [2, 3],
              'min_samples_leaf': [3, 4, 5],
              'min_samples_split': [8, 10, 12],}

# run grid search
# random_search = GridSearchCV(GradientBoostingRegressor, param_grid=param_grid, n_jobs=-1)

model_args = [
    [DecisionTreeRegressor, 'dt_reg', {'max_depth': 5}], 
    [RandomForestRegressor, 'rf_reg', {'n_jobs': -1}], 
#     [MultiOutputRegressor, 'gb_reg', {'n_jobs':-1, 'estimator': GradientBoostingRegressor(max_depth=10)}],
#     [GridSearchCV, 'cv_gb', {'estimator': GradientBoostingRegressor(max_depth=10), 'param_grid': param_grid, 'n_jobs':-1}]
]



In [18]:
for model in model_args:
    model_fun, model_str, extra_args = model
    
    print('Running:', model_str)
    for iter, (i, o) in enumerate(experiments):
        print(f'{iter}: Input variables = {i}; output variables = {o}')
        
        X_train, y_train, X_test, y_test, time = create_X_y_time(input_vars=i, 
                                                                 output_vars=o, 
                                                                 data=[data_train, data_test], 
                                                                 lead_time=lead_time, 
                                                                 data_subsample=5)
        
        data_needed_train = [X_train, y_train, X_test, y_test, nlat, nlon]
        
        predictions, model_res, mse_train, mse_test = train_regression(model=model_fun,
                                                                       data=data_needed_train,
                                                                       num_outputs=len(o),
                                                                       extra_args=extra_args,
                                                                       verbose=False)
        
        predictions = unnormalize_preds(preds=predictions, 
                                        output_vars=o, 
                                        valid_time=time, 
                                        lat_lon=[data_train.lat, data_train.lon],
                                        data_std=data_std, 
                                        data_mean=data_mean)

        preds[model_str].append(predictions)
        models[model_str].append(model_res)
        
        # Compute Weighted RMSE: RMSE with a weight that takes into account grid location
        r = compute_weighted_rmse(predictions, data_test).compute()
        if verbose:
            print('; '.join([f'{v} = {r[v].values}' for v in r]) + '\n')
        
        train_mse[model_str].append(mse_train)
        test_mse[model_str].append(mse_test)
        weighted_mse[model_str].append(r)
        
        # Save models
        preds_nc_str  = f'{results}/{model_str}_{lead_time_str}_{"_".join(i)}_{"_".join(o)}.nc'
        model_res_str = f'{results}/{model_str}_{lead_time_str}_{"_".join(i)}_{"_".join(o)}.pkl'
        
        predictions.to_netcdf(preds_nc_str)
        to_pickle(model_res, model_res_str)
    print('----------------------------------------------------------------------------')

Running: dt_reg
0: Input variables = ['t']; output variables = ['t']
1: Input variables = ['z', 't']; output variables = ['t']
2: Input variables = ['z', 't', 't2m']; output variables = ['t']
3: Input variables = ['t2m']; output variables = ['t2m']
4: Input variables = ['z', 't']; output variables = ['t2m']
5: Input variables = ['z', 't', 't2m']; output variables = ['t2m']
----------------------------------------------------------------------------
Running: rf_reg
0: Input variables = ['t']; output variables = ['t']
1: Input variables = ['z', 't']; output variables = ['t']
2: Input variables = ['z', 't', 't2m']; output variables = ['t']


_call non-retriable exception: 
Traceback (most recent call last):
  File "/opt/tljh/user/lib/python3.7/site-packages/gcsfs/core.py", line 487, in _call
    validate_response(r, path)
  File "/opt/tljh/user/lib/python3.7/site-packages/gcsfs/core.py", line 132, in validate_response
    raise HttpError({"code": r.status_code})
gcsfs.utils.HttpError
_call non-retriable exception: 
Traceback (most recent call last):
  File "/opt/tljh/user/lib/python3.7/site-packages/gcsfs/core.py", line 487, in _call
    validate_response(r, path)
  File "/opt/tljh/user/lib/python3.7/site-packages/gcsfs/core.py", line 132, in validate_response
    raise HttpError({"code": r.status_code})
gcsfs.utils.HttpError
_call non-retriable exception: 
Traceback (most recent call last):
  File "/opt/tljh/user/lib/python3.7/site-packages/gcsfs/core.py", line 487, in _call
    validate_response(r, path)
  File "/opt/tljh/user/lib/python3.7/site-packages/gcsfs/core.py", line 132, in validate_response
    raise HttpError(

HttpError: 

In [ ]:
import pandas as pd

results_df = pd.DataFrame(columns=['model', 'input', 'output',  'train_mse', 'test_mse', 'weighted_mse'])

block_rows = []
for key in models.keys():
    first_three = np.hstack((np.array([key] * len(experiments)).reshape(len(experiments),1), np.array(experiments, dtype='object')))
    block_rows.append(first_three)
    
block_rows = np.array(block_rows).reshape(len(models.keys()) * len(experiments), 3) 

results_df['model'] = block_rows[:, 0]
results_df['input'] = block_rows[:, 1]
results_df['output'] = block_rows[:, 2]



train_mse_col = []
for model, ds_list in train_mse.items():
    train_mse_col += ds_list

test_mse_col = []
for model, ds_list in test_mse.items():
    test_mse_col += ds_list
    

weighted_mse_col = []
for model, ds_list in weighted_mse.items():
    model_metrics = []

    for ds in ds_list:
        exp_metrics = []
        for var in list(ds.variables):
            exp_metrics.append(round(float(ds[var].values), 4))

        model_metrics.append(exp_metrics)
    weighted_mse_col += model_metrics

results_df['train_mse'] = train_mse_col
results_df['test_mse'] = test_mse_col

results_df['weighted_mse'] = weighted_mse_col
results_df['weighted_mse'] = results_df['weighted_mse'].apply(lambda x : x[0])

results_df['output'] = results_df['output'].apply(lambda x: x[0])

results_df

In [ ]:
import seaborn as sns
sns.set_style('darkgrid')

def get_indexes(get_var):
    idxs = []
    for idx, out in results_df['output'].items():
        for var in out:
            var = str(var).lower()

            if get_var == var:
                idxs.append(idx)
    return idxs

idxs = get_indexes('t2m')
t2m_out = results_df[results_df['output'] == 't2m']

f, axs = plt.subplots(nrows=1, ncols=3, figsize=(18, 5))

sns.lineplot(data=t2m_out, x='model', y='train_mse', ax=axs[0])
sns.lineplot(data=t2m_out, x='model', y='test_mse',  ax=axs[1])
sns.lineplot(data=t2m_out, x='model', y='weighted_mse', ax=axs[2])
f.show()


idxs = get_indexes('t')
t_out = results_df[results_df['output'] == 't']

f, axs = plt.subplots(nrows=1, ncols=3, figsize=(18, 5))

sns.lineplot(data=t_out, x='model', y='train_mse', ax=axs[0])
sns.lineplot(data=t_out, x='model', y='test_mse',  ax=axs[1])
sns.lineplot(data=t_out, x='model', y='weighted_mse', ax=axs[2])

f.show()

In [ ]:
f, axs = plt.subplots(nrows=1, ncols=3, figsize=(18, 5))

sns.lineplot(data=results_df, x='model', y='train_mse', hue='output', ax=axs[0])
sns.lineplot(data=results_df, x='model', y='test_mse', hue='output', ax=axs[1])
sns.lineplot(data=results_df, x='model', y='weighted_mse', hue='output', ax=axs[2])
f.show()


In [ ]:
import xgboost as xgb

In [ ]:
# import xgboost as xgb # https://xgboost.readthedocs.io/en/latest/python/python_api.html

# preds_xgb_reg= []
# models_xgb_reg = []

for iter, (i, o) in enumerate(experiments):
    print(f'{iter}: Input variables = {i}; output variables = {o}')

    X_train, y_train, X_test, y_test, time = create_X_y_time(input_vars=i, 
                                                             output_vars=o, 
                                                             data=[data_train, data_test], 
                                                             lead_time=lead_time, 
                                                             data_subsample=5)

    data_needed_train = [X_train, y_train, X_test, y_test, nlat, nlon]

    predictions, model_res, mse_train, mse_test = train_regression(model=MultiOutputRegressor,
                                                                   data=data_needed_train,
                                                                   num_outputs=len(o),
                                                                   extra_args={
                                                                       'estimator': xgb.XGBRegressor(objective='reg:linear'),
                                                                       'n_jobs':-1
                                                                   },
                                                                   verbose=False)

    predictions = unnormalize_preds(preds=predictions, 
                                    output_vars=o, 
                                    valid_time=time, 
                                    lat_lon=[data_train.lat, data_train.lon],
                                    data_std=data_std, 
                                    data_mean=data_mean)

    preds[model_str].append(predictions)
    models[model_str].append(model_res)

    # Compute Weighted RMSE: RMSE with a weight that takes into account grid location
    r = compute_weighted_rmse(predictions, data_test).compute()
    if verbose:
        print('; '.join([f'{v} = {r[v].values}' for v in r]) + '\n')

    train_mse[model_str].append(mse_train)
    test_mse[model_str].append(mse_test)
    weighted_mse[model_str].append(r)

    # Save models
    preds_nc_str  = f'{results}/{model_str}_{lead_time_str}_{"_".join(i)}_{"_".join(o)}.nc'
    model_res_str = f'{results}/{model_str}_{lead_time_str}_{"_".join(i)}_{"_".join(o)}.pkl'

    predictions.to_netcdf(preds_nc_str)
    to_pickle(model_res, model_res_str)
print('----------------------------------------------------------------------------')